In [ ]:
!nvidia-smi


Wed Dec 11 17:09:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0              50W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install cvzone
!pip install mediapipe
!pip install tensorflow

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26298 sha256=f985026ac36c2af042b212b3f86e8945cfe921f9ee9c523dd47d07cb5a122bac
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 50.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from sklearn.metrics import confusion_matrix, classification_report




In [ ]:
# Check if GPU is being used
if tf.test.gpu_device_name():
    print(f"TensorFlow is using GPU: {tf.test.gpu_device_name()}")
else:
    print("TensorFlow is not using the GPU.")

TensorFlow is using GPU: /device:GPU:0


In [ ]:
#paths
michael_data_folder = "/content/drive/MyDrive/50.035 Computer Vision Group Assignment/Data (Combined)/cropped (Mitchel & Ad)/"
landmarks_only_folder = "/content/drive/MyDrive/50.035 Computer Vision Group Assignment/Data (Combined)/cropped (Mitchel & Ad)/Landmarks (black)"

data_folder = landmarks_only_folder #note to nada: this path currently points at landmarks only, feel free to change if you want to train on diff dataset

In [ ]:
#create csv - note: u only need to run this once btw if u keep retraining the model
csv_path = os.path.join(michael_data_folder,'_landmarks_metadata.csv') #note to nada: change this if ur not training landmarks only
data = []

for filename in os.listdir(data_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        label = filename.split(" (")[0]
        img_path = os.path.join(data_folder, filename)
        data.append({"img_path": img_path, "label": label})

df = pd.DataFrame(data)
print(df.head())

unique_labels = df['label'].unique()
label_map = {label: idx for idx, label in enumerate(unique_labels)}
df['numeric_label'] = df['label'].map(label_map)
df = df.drop(columns='label')

df.to_csv(csv_path, index=False)
print("DataFrame created and saved successfully!")



                                            img_path  label
0  /content/drive/MyDrive/50.035 Computer Vision ...  tur_N
1  /content/drive/MyDrive/50.035 Computer Vision ...  tur_N
2  /content/drive/MyDrive/50.035 Computer Vision ...  tur_N
3  /content/drive/MyDrive/50.035 Computer Vision ...  tur_N
4  /content/drive/MyDrive/50.035 Computer Vision ...  tur_N
DataFrame created and saved successfully!


In [ ]:
# Load the metadata CSV
metadata = pd.read_csv(csv_path)

# Ensure the 'img_path' column contains full paths to images
metadata['img_path'] = metadata['img_path'].apply(lambda x: os.path.join(data_folder, x))
metadata.head()

,img_path,numeric_label
0,/content/drive/MyDrive/50.035 Computer Vision ...,0
1,/content/drive/MyDrive/50.035 Computer Vision ...,0
2,/content/drive/MyDrive/50.035 Computer Vision ...,0
3,/content/drive/MyDrive/50.035 Computer Vision ...,0
4,/content/drive/MyDrive/50.035 Computer Vision ...,0


In [ ]:
# Split the metadata into training, validation, and test sets
train_df = metadata.sample(frac=0.8, random_state=42)  # 80% training
remaining_df = metadata.drop(train_df.index)
val_df = remaining_df.sample(frac=0.5, random_state=42)  # 50% of remaining = 10% overall
test_df = remaining_df.drop(val_df.index)  # Remaining 10% for test

# Convert 'numeric_label' to string for compatibility with class_mode='categorical'
train_df['numeric_label'] = train_df['numeric_label'].astype(str)
val_df['numeric_label'] = val_df['numeric_label'].astype(str)
test_df['numeric_label'] = test_df['numeric_label'].astype(str)



# # Create ImageDataGenerator instances
# train_datagen = ImageDataGenerator(
#     rescale=1.0 / 255.0,  # Normalize pixel values
#     validation_split=0.2
# )
# val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# train_generator = train_datagen.flow_from_dataframe(
#     train_df,
#     x_col='img_path',
#     y_col='numeric_label',
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',
#     shuffle=True
# )

# val_generator = val_datagen.flow_from_dataframe(
#     val_df,
#     x_col='img_path',
#     y_col='numeric_label',
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',
#     shuffle=False
# )



In [ ]:
# Paths and labels from the DataFrame
train_paths = train_df['img_path'].values
train_labels = tf.keras.utils.to_categorical(train_df['numeric_label'].astype(int).values)
val_paths = val_df['img_path'].values
val_labels = tf.keras.utils.to_categorical(val_df['numeric_label'].astype(int).values)
test_paths = test_df['img_path'].values
test_labels = tf.keras.utils.to_categorical(test_df['numeric_label'].astype(int).values)

# Preprocessing function for loading and resizing images
def preprocess_image(file_path, label):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize to target size
    image = image / 255.0  # Normalize pixel values
    return image, label

# Create tf.data.Dataset objects
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

# Optimize datasets
batch_size = 32
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
# Load ResNet50 and modify it for the dataset
num_classes = 48

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

for layer in model.layers[-10:]:  # unfreeze last 10 layers
    layer.trainable = True


model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset, epochs=20)

Epoch 1/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 293s 100ms/step - accuracy: 0.2284 - loss: 3.1700 - val_accuracy: 0.1862 - val_loss: 3.2771
Epoch 2/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 49ms/step - accuracy: 0.9423 - loss: 0.2772 - val_accuracy: 0.9257 - val_loss: 0.4000
Epoch 3/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 48ms/step - accuracy: 0.9811 - loss: 0.0921 - val_accuracy: 0.9174 - val_loss: 0.3093
Epoch 4/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - accuracy: 0.9866 - loss: 0.0717 - val_accuracy: 0.9331 - val_loss: 0.2562
Epoch 5/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - accuracy: 0.9869 - loss: 0.0558 - val_accuracy: 0.9833 - val_loss: 0.0846
Epoch 6/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - accuracy: 0.9846 - loss: 0.0587 - val_accuracy: 0.9718 - val_loss: 0.1057
Epoch 7/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - accuracy: 0.9934 - loss: 0.0348 - val_accuracy: 0.9885 - val_loss: 0.0690
Epoch 8/20
239/239 ━━━━━━━━━━━━━━━━━━━━ 13s 47ms/step - accuracy: 0.9856 - loss: 0.0544 

In [ ]:
# evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Set Accuracy: {test_accuracy}")


30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 166ms/step - accuracy: 0.9954 - loss: 0.0250
Test Set Accuracy: 0.9979057312011719


In [ ]:
def get_labels(dataset):
    labels = []
    for _, label in dataset:
        labels.append(label.numpy())
    return np.concatenate(labels)

y_true = get_labels(test_dataset)

y_pred = np.argmax(model.predict(test_dataset), axis=1)

# ensure y_true is a 1D array of class labels (not one-hot encoded)
y_true = np.argmax(y_true, axis=1) if y_true.ndim > 1 else y_true

cm = confusion_matrix(y_true, y_pred)
class_names = list(label_map.keys())


# Plot confusion matrix
def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(10, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

plot_confusion_matrix(cm, class_names)
plt.show()



In [ ]:
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

In [ ]:
# Compute F2 score
def fbeta_score(y_true, y_pred, beta=2):
    from sklearn.metrics import precision_recall_fscore_support
    precision, recall, _, _ = precision_recall_fscore_support(y_true, y_pred, beta=beta, average='weighted')
    fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall + 1e-7)
    return fbeta

f2_score = fbeta_score(y_true, y_pred, beta=2)
print(f"F2 Score: {f2_score}")

In [ ]:
model_path = os.path.join(michael_data_folder,'models/resnet_landmarks_only.h5') #note: change this if ur making a copy of the file!!!
model.save(model_path)


In [ ]:
print(label_map)